# **Data Analysis**

## **Descriptive Data Analysis**

## **0. Imports**

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

### **0.1. Data Collection** 

In [2]:
path = '/home/eron/repos/SalesPricePredict/'
database_name = 'database_hm.sqlite'
conn = create_engine('sqlite:///' + path + database_name, echo = False)

In [3]:
query = """
    SELECT * FROM vitrine
"""

In [4]:
df_raw = pd.read_sql(query, con = conn)

## **1. Data Description**

In [5]:
df01 = df_raw.copy()

### **1.1. Data Dimension**

In [6]:
print('Number of rows: {}'.format(df01.shape[0]))
print('Number of rows: {}'.format(df01.shape[1]))

Number of rows: 916
Number of rows: 14


### **1.2. Data Types**

In [7]:
df01.dtypes

product_id          object
style_id            object
color_id            object
product_name        object
color_name          object
fit                 object
product_price      float64
size_number         object
size_model          object
cotton             float64
polyester          float64
elastane           float64
elasterell         float64
scrapy_datetime     object
dtype: object

In [8]:
# Convert object to datetime
df01['scrapy_datetime'] = pd.to_datetime(df01['scrapy_datetime'])

In [9]:
df01.dtypes

product_id                 object
style_id                   object
color_id                   object
product_name               object
color_name                 object
fit                        object
product_price             float64
size_number                object
size_model                 object
cotton                    float64
polyester                 float64
elastane                  float64
elasterell                float64
scrapy_datetime    datetime64[ns]
dtype: object

### **1.3. Missing Data**

In [10]:
df01.isna().sum()

product_id           0
style_id             0
color_id             0
product_name         0
color_name           0
fit                  0
product_price        0
size_number        721
size_model         685
cotton               0
polyester            0
elastane             0
elasterell           0
scrapy_datetime      0
dtype: int64

In [11]:
df01.isna().sum() / df01.shape[0]

product_id         0.000000
style_id           0.000000
color_id           0.000000
product_name       0.000000
color_name         0.000000
fit                0.000000
product_price      0.000000
size_number        0.787118
size_model         0.747817
cotton             0.000000
polyester          0.000000
elastane           0.000000
elasterell         0.000000
scrapy_datetime    0.000000
dtype: float64

### **1.4. Solving Missing Data**

After reviewing the ETL code and visiting the company's website, I noticed that the **size number** and **size model** data are not reported in the source, so I decided to **exclude the features**.

In [12]:
df01 = df01.drop(columns = ['size_number', 'size_model'])

### **1.5. Data Description**

In [13]:
num_attributes = df01.select_dtypes(include = ['float64', 'int64'])
cat_attributes = df01.select_dtypes(exclude = ['float64', 'int64', 'datetime64[ns]'])

#### **1.5.1. Numerical Data**

In [14]:
# Central tendency - mean and median
t1 = pd.DataFrame(num_attributes.apply(np.mean)).T
t2 = pd.DataFrame(num_attributes.apply(np.median)).T

# Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(np.min)).T
d3 = pd.DataFrame(num_attributes.apply(np.max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

# Concat
m1 = pd.concat([d2, d3, d4, t1, t2, d1, d5, d6]).T.reset_index()
m1.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m1

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,product_price,5.99,49.99,44.00,26.313144,24.99,9.909875,0.568862,-0.216318
1,cotton,0.66,1.00,0.34,0.962969,0.98,0.063127,-2.439100,5.463768
2,polyester,0.00,1.00,1.00,0.175404,0.00,0.284468,1.384655,0.422636
3,elastane,0.00,0.02,0.02,0.005502,0.00,0.007935,0.983963,-0.702336
4,elasterell,0.00,0.09,0.09,0.005524,0.00,0.020332,3.416728,9.705184


##### **Minimum**

- Menor valor do conjunto de dados

##### **Maximum**

- Maior valor do conjunto de dados

##### **Range**

- Diferença entre o máximo e o mínimo. 
- Um range maior implica em dados mais dispersos.

##### **Mean**

- Resumo de um conjunto de dados em um único ponto, o centro de uma nuvem de pontos. 
- Soma de todos os valores dividido pela quantidade de valores. 
- Sofre forte impacto dos outliers.

##### **Median**

- Número na posição central de uma lista ordenada. Quando o conjunto de dados tiver uma quantidade par de elementos, a mediana será a média entre os dois pontos centrais.
- Não sofre o impacto dos outliers, pois independe da grandeza dos valores.

##### **Std**

- Estimativa de variabilidade ou dispersão.
    - Mede os valores do conjunto de dados que estão próximos ou distantes em torno de um ponto central.
- Desvio: Diferença entre os valores observados e a estimativa de localização (média ou mediana). Mostra quão dispersos estão os dados em torno da média.
- Variância: É a soma dos desvios elevados ao quadrado, para tornar positivo os números negativos, divididos pelo número de valores do conjunto de dados menos 1 (n -1). D
- Desvio padrão: É a raíz quadrada da variância. A raiz faz com que o resultado volte para a escala original, após serem elevados ao quadrado para o cálculo da variância.

##### **Skewness**

- É a medida de quanto a distribuição dos dados está assimétrica em relação ao centro, à média.
- A distribuição "normal" não possui skewness (skewness = 0)
- Skewness positiva:
     - Se a mediana é menor que a média, há um deslocamento de dados para a esquerda, portanto a skewness será positiva.
- Skewness 0: 
    - Se a média é igual a mediana, não há deslocamento dos dados, é uma distribuição "normal".
- Skewness negativa:
    - Se a mediana é maior que a média, há um deslocamento de dados para a direita, portanto a skewness será negativa.
    
- Interpretação:
    - A skewness mostra a concentração dos dados de um lado da média.
    - Algoritmos de ML precisam treinar com dados não enviesados, ou seja, a distribuição da variável resposta (alvo) precisa ser o mais próximo de uma distribuição normal.

##### **Kurtosis**

- Descreve a distribuição dos dados em torno da média.
- Evidencia uma distribuição de caudas longas (probabilidade de acontecer valores extremos).
- Kurtosis alta significa grande concentração ao redor da média, situação ruim pois os modelos podem aprender apenas valores próximos da média e ignorar outros acontecimentos.

#### **Momentos da estística:**

- Momento 1: Estimativa de localização (Média ou Mediana)
- Momento 2: Estimativa de Variabilidade (Desvios, Variância, Desvio Padrão, Intervalo)
- Momento 3: Skewness (Assimetria da distribuição dos dados em relação a média)
- Momento 4: Kurtosis (Evidência de caudas longas - probabilidade acontecerem valores extremos)

#### **1.5.2. Categorical Data**

### **2. Feature Engineering**

### **3. Data Filtering**

### **4. EDA - Exploratory Data Analysis**